In [407]:
import pandas as pd
import numpy as np
from modules import *
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk import word_tokenize
from konlpy.tag import Okt
from nltk.util import everygrams

In [408]:
#원하는 태그 추출 & 불용어 제거
okt = Okt()

#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다",
                 '에서','고','이다','젛','뇨','껀데','뭘'
                 ,'최고','느리다','마을','최고','항상','포장','재다','말다','목','넘김','만족하다','정말','역시','아주','감사하다','오다','자주','요','늘','넘다','않다','더','병','다','숙성','나다','술','드리다','마트','꼼꼼하다','잇다'
                 ,'복','순도','용','비싸다','가격','많다','강하다','보내다','추천','보고','쏘다','느낌','사다','처음','많이','톡','생각','엄청','날','아니다'
                 ,'믿다', '구입', '사먹다', '한번', '싶다', '콤', '완전', '제품', '재', '해주다', '말', '사람','특별하다','언제나','빨르다','두번째','전','계속','달'
                 ,'추석', '인생', '최애', '명절', '해', '중', '생각나다', '시원하다', '다르다', '알다', '가족', '정도', '기대', '친구', '괜찮다', '부모님', '지인', '아직'
                 ,'만족스럽다', '빼다', '상품', '굿', '못', '도', '기분', '다시', '주다', '즐기다', '맞다', '맛보다', '제일', '집', '입맛', '모르다', '먹기'
                 ,'조심하다', '따다', '차다', '엄마', '예정', '만족', '감', '일반', '파다', '막', '근데', '별로', '적당하다','예전', '가끔', '유명하다', '만', '뚜껑', '매우', '기대하다', '의사', '한잔', '신선하다', '느끼다', '그냥', '아쉽다'
                 ,'오픈', '쎄다', '여자', '호불호', '꼭', '벌써', '버리다', '이렇게', '돼다', '크다', '느껴지다','고급', '종종', '어울리다', '알', '남편', '빨리', '어른', '소문', '아버지', '마시기', '할인', '살다' 
                 ,'풍부하다', '모임', '해드리다', '빠지다', '나서다', '원래', '아빠', '걸리다', '안', '도착', '없이'
                 ,'하루', '딱이다', '상태', '향', '후', '설', '주변', '사서', '때문', '반하다', '연말'
                 ,'해봤다', '강추', '찾다', '자꾸', '살짝', '몇번', '빨', '요즘', '신세계', '사진', '생일'
                 ,'빠르다', '자다', '구매', '마시다', '받다', '보다', '없다', '새다', '숙취', '아스파탐', '대포', '넘어가다','주가'
                 ,'지나다', '무', '머리', '편하다', '아프다', '두다', '인터넷', '택배', '숙성하다', '일주일', '봄', '배달', '만들다', '애용', '냉장고' 
                 ,'여름', '잔', '앞', '며칠', '팔다', '달라지다', '가장', '변하다', '번창', '안전하다', '젤', '배상면', '건강하다', '굿굿'
                 ,'따르다', '이용', '확실하다', '날짜', '시중', '들어가다', '첨가', '유통', '아이스팩', '제조', '넣다', '생', '다양하다','종류','들어가다','전통주','마켓','술술','후기','사보다','부담','여러가지','리뷰','블루'
                 ,'인기','궁금하다','대상','무난','취향','적다','단','평이','이용','터지다','맘'
                 ]
                 +'뒤 비비 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 기한 달라 첫날 첨 주말 걱정 전이 이유 해보다 저번 보지 함 기대다 상 비 베스트 도수 음 존맛 개인'.split()
                 ) #불용어 추가
stopwords=set(stopwords) # 중복제거    

# 형태소 분석 함수 만들기
def okt_pos_tagging2(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    # n_gram 만들기
    egram = list(everygrams(words, min_len=1, max_len=1))
    egram_token = [' '.join(grams) for grams in egram]

    return egram_token

def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag 
             in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]

    return words


In [409]:
# 불러올 csv 이름 리스트
# 전체 리뷰를 불러온후 tf-idf 분석
name_list =["술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"]

In [410]:
all_data = pd.DataFrame()

for name in name_list:
    data = pd.read_csv(f"{name}.csv", encoding='utf-8-sig')
    # 전처리
    data.drop_duplicates(['text'],ignore_index=True, inplace=True) # 중복 제거
    try:
        data.columns = ['date', 'star','doc']
    except:
        data.columns = ['date', 'star','doc','name']
    data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
    data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
    data.doc = data.doc.str.replace("럽고목","럽고 목")
    data.doc = data.doc.str.replace("부 드럽","부드럽")
    data.doc = data.doc.str.replace("가성", "가성비")
    data.dropna(inplace=True)
    data.reset_index(drop=True, inplace=True)
    # globals()["감홍로 40도"] 등으로 데이터 프레임 저장됨
    globals()[f"{name}"] = data
    all_data = pd.concat([all_data,data],axis=0)


C:\Users\user\AppData\Local\Temp\ipykernel_13716\421399407.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\421399407.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\421399407.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\421399407.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data.doc = data.doc.str.replace("\s+", " ") # white space 삭제
C:\Users\user\AppData\Local\Temp\ipykernel_13716\421399407.py:11: FutureWarning: The

In [411]:
all_data

,date,star,doc
0,2022-10-31,5,지인으로부터 알게된 복순도가 막걸리처음 먹는순간 너무 맛있어서 깜짝놀랐어요원래 술을...
1,2022-11-17,5,그냥 차게 마셔도 군내나 잔여물 찌꺼기 느낌이 거의 없이 깔끔담백하고 샴페인처럼 톡...
2,2022-10-19,5,지인이 비싼데 고급지고 맛있는 막걸리가있다고 해서하도 궁금하여 한번 시켜봤어요배송도...
3,2022-10-28,3,본인은 맛있었는데 아버지는 너무 삭힌거같다는 평이네요 새콤함이 좀 있긴함근데 막걸리...
4,2022-10-13,5,복순도가 맛있다고 한참전에 소문만 들었지 먹어본것은 올해가 처음이네요 평소에도 달달...
...,...,...,...
5674,2020-04-03,5,다양한술을 집에서 편하게 주문가능하기에 자주 주문합니다이번에도 즐겁게 마시겠읍니다
5675,2020-04-03,5,빠른 배송 감사합니당
5676,2020-03-31,5,단맛은 있는데 꿀 단맛이어서 나쁘지 않아요 기분 나쁘지 않은 단맛이에요 인공감미료가...
5677,2020-03-26,5,맛이 달고 청량감이 좋아요


In [412]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_pos_tagging)
tfidf_fit = tfidf.fit(all_data.doc)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [413]:
# 특징을 추출할 문서의 tf-idf를 구함
# "술 리뷰(복순도가)" : 110,"느린마을 막걸리" : 130,"술 리뷰(죽향도가)" : 300
a = "술 리뷰(죽향도가)"
bow_rep_tfidf = tfidf_fit.transform(globals()[a].doc)
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names(),
    'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
    'idf': tfidf.idf_
})

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [414]:
# idf 가 높을수록 잘 안나오는 단어
# tf-idf 가 높을수록 중요한 단어
# 두값을 적절히 조정해서 특징 추출

word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a]))>300]

C:\Users\user\AppData\Local\Temp\ipykernel_13716\693789458.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a]))>300]


,단어,tf-idf,idf
1838,달달,319.130357,4.229926
1095,깔끔하다,136.022831,4.472532
1205,꿀,122.093401,6.140744
4687,선물,103.959862,3.524104
1836,달다,102.059033,5.006997
1814,단맛,84.695748,5.024410
59,가볍다,66.650113,5.718813
1213,꿀맛,51.048639,6.807425
1875,달콤하다,40.402541,5.716303


In [424]:
# 특징을 추출할 문서의 tf-idf를 구함
a = ["술 리뷰(복순도가)","느린마을 막걸리","술 리뷰(죽향도가)"]
Tags = {"boksundoga": [],"maeul": [],"jukhyang": []}
keys = ["boksundoga","maeul","jukhyang"]
for i in range(len(a)):
    bow_rep_tfidf = tfidf_fit.transform(globals()[a[i]].doc)
    word_count = pd.DataFrame({
        '단어': tfidf.get_feature_names(),
        'tf-idf': bow_rep_tfidf.sum(axis=0).flat,
        'idf': tfidf.idf_
    })
    
    df = word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>110]

    word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>110].to_csv(f'막걸리_특징{i}.csv')

    # for i in range(len(Tags)):
    for j in df['단어']:
        Tags[keys[i]].append(j)
Tags['boksundoga']

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\user\AppData\Local\Temp\ipykernel_13716\1894880630.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>110]
C:\Users\user\AppData\Local\Temp\ipykernel_13716\1894880630.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  word_count.sort_values('tf-idf',ascending=False).head(50)[word_count['idf']>3.5][word_count['tf-idf'] * (len(all_data)/len(globals()[a[i]]))>110].to_csv(f'막걸리_특징{i}.csv')


['선물',
 '샴페인',
 '달달',
 '깔끔하다',
 '와인',
 '시큼하다',
 '스파클링',
 '달다',
 '신맛',
 '상큼',
 '청량감',
 '고급스럽다',
 '달콤하다',
 '저렴하다',
 '가볍다',
 '단맛',
 '요구르트',
 '진하다',
 '산미',
 '사이다',
 '청량하다']

In [425]:
Tags['maeul']

['깔끔하다', '선물', '달달', '달다', '우유', '단맛', '순하다', '진하다', '저렴하다', '달콤하다']

In [417]:
df0 = pd.read_csv('막걸리_특징0.csv')
del df0['Unnamed: 0']
df0

,단어,tf-idf,idf
0,선물,1335.804490,3.524104
1,샴페인,274.373651,5.257959
2,달달,203.476793,4.229926
3,깔끔하다,163.048610,4.472532
4,와인,151.300152,5.625332
5,시큼하다,147.637158,5.691550
6,스파클링,141.737922,5.900130
7,달다,126.697367,5.006997
8,신맛,123.166654,5.894124
9,상큼,107.322048,6.125535


In [418]:
df1 = pd.read_csv('막걸리_특징1.csv')
del df1['Unnamed: 0']
df1

,단어,tf-idf,idf
0,깔끔하다,340.813924,4.472532
1,선물,270.808152,3.524104
2,달달,250.087509,4.229926
3,달다,145.121003,5.006997
4,우유,138.316239,5.906172
5,단맛,137.852039,5.024410
6,순하다,107.852406,6.088494
7,진하다,99.839816,5.781009
8,저렴하다,69.016782,5.988237
9,달콤하다,57.597498,5.716303


In [419]:
df2 = pd.read_csv('막걸리_특징2.csv')
del df2['Unnamed: 0']
df2

,단어,tf-idf,idf
0,달달,319.130357,4.229926
1,깔끔하다,136.022831,4.472532
2,꿀,122.093401,6.140744
3,선물,103.959862,3.524104
4,달다,102.059033,5.006997
5,단맛,84.695748,5.024410
6,가볍다,66.650113,5.718813
7,꿀맛,51.048639,6.807425
8,달콤하다,40.402541,5.716303
9,저렴하다,35.261643,5.988237


In [427]:
def getHashTagString(list_of_tags):
	hashTagString = ""

	for tag in list_of_tags:
		hashTagString += " #" + tag

	return hashTagString

In [428]:
getHashTagString(Tags['boksundoga'])[1:13]

'#선물 #샴페인 #달달'

In [431]:
getHashTagString(Tags['maeul'])[1:14]

'#깔끔하다 #선물 #달달'

In [434]:
getHashTagString(Tags['jukhyang'])[1:13]

'#달달 #깔끔하다 #꿀'